# **Toronto Postal Codes - Wikipedia PART-1**

In [1]:
import pandas as pd
import numpy as np

# Parse HTML

In [45]:
canada_suburbs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
canada_suburbs.head(2)

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...


In [44]:
canada_suburbs.shape[0]
postal_code = []
Borough = []
Neighborhood = []
for row in range(canada_suburbs.shape[0]):
  for col in range(canada_suburbs.shape[1]):
    if 'Not assigned' not in canada_suburbs.iloc[row][col]:
      #print(canada_suburbs.iloc[row][col])
      borough_start_index = canada_suburbs.iloc[row][col].index('(')
      borough_end_index = canada_suburbs.iloc[row][col].index(')')
      Borough.append(canada_suburbs.iloc[row][col][borough_start_index+1:borough_end_index])
      Neighborhood.append(canada_suburbs.iloc[row][col][3:borough_start_index])
      postal_code.append(canada_suburbs.iloc[row][col][:3])
toronto_suburbs = pd.DataFrame(np.column_stack([postal_code,Neighborhood, Borough]), 
                               columns=['Postal Code', 'Neighborhood', 'Borough'])
toronto_suburbs.head(10)

,Postal Code,Neighborhood,Borough
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [46]:
toronto_suburbs.shape

(103, 3)

# **Toronto Postal Codes - Wikipedia PART-2**

In [ ]:
!pip install geocoder
import geocoder

In [50]:
toronto_suburbs['Latitude'] = None
toronto_suburbs['Longitude'] = None
toronto_suburbs.head(5)

,Postal Code,Neighborhood,Borough,Latitude,Longitude
0,M3A,North York,Parkwoods,None,None
1,M4A,North York,Victoria Village,None,None
2,M5A,Downtown Toronto,Regent Park / Harbourfront,None,None
3,M6A,North York,Lawrence Manor / Lawrence Heights,None,None
4,M7A,Queen's Park,Ontario Provincial Government,None,None


In [55]:
for row, postal_code in enumerate(toronto_suburbs['Postal Code']):
    coordinates = None
    
    while(coordinates is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        coordinates = g.latlng
    
    if coordinates:
        latitude = coordinates[0]
        longitude = coordinates[1]
    
    toronto_suburbs.loc[row, 'Latitude'] = latitude
    toronto_suburbs.loc[row, 'Longitude'] = longitude

toronto_suburbs.head(10)

,Postal Code,Neighborhood,Borough,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7525,-79.3299
1,M4A,North York,Victoria Village,43.7306,-79.3131
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.6551,-79.3626
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.7233,-79.4504
4,M7A,Queen's Park,Ontario Provincial Government,43.6625,-79.3919
5,M9A,Etobicoke,Islington Avenue,43.6626,-79.5283
6,M1B,Scarborough,Malvern / Rouge,43.8114,-79.1966
7,M3B,North York,Don Mills,43.7492,-79.3619
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.7072,-79.3119
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6574,-79.378


# **Toronto Postal Codes - Wikipedia PART-3**

In [57]:
!pip install geopy
from geopy.geocoders import Nominatim
!pip install folium
import folium
address = 'Toronto, CA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


# Create Toronto Map

In [60]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(toronto_suburbs['Latitude'], toronto_suburbs['Longitude'], toronto_suburbs['Borough'], toronto_suburbs['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Group by Borough

In [62]:
toronto_suburbs.groupby(by='Borough').count().sort_values(by='Neighborhood', ascending=False)

,Postal Code,Neighborhood,Latitude,Longitude
Borough,,,,
Downsview,4,4,4,4
Don Mills,2,2,2,2
Willowdale,2,2,2,2
Agincourt,1,1,1,1
Parkwoods,1,1,1,1
...,...,...,...,...
Forest Hill North & West,1,1,1,1
First Canadian Place / Underground city,1,1,1,1
Fairview / Henry Farm / Oriole,1,1,1,1
